In [ ]:
from cornac.data import Reader
from cornac.datasets import movielens
from cornac.data import Dataset, FeatureModality
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.metrics import RMSE
from cornac.models import  ItemKNN, UserKNN, NMF, BPR, MF,VAECF, NeuMF
# from MF import MF
import pandas as pd
import numpy as np
import cornac
import math
# import seaborn as sns
# import matplotlib.pyplot as plt

: 

In [ ]:
rating_data_pd = pd.read_csv(
    "../data/ml-100K/indexed_interactions.csv",
    sep="\t",
    header=0,
    names=["userID", "itemID", "Rating", "Timestamp"],
)
rating_data = rating_data_pd.to_numpy()
rating_data.__len__()
rating_data_pd

In [ ]:

movie = '../data/ml-100K/i_id_mapping_genre.csv'

df_movie = pd.read_csv(movie, sep='\t')
print(f'shape: {df_movie.shape}')


df_movie.columns = ["item_id", "Name","genres","itemID"]
df_movie[:4]
movies = df_movie.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0
    
for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1
movies


movies

In [ ]:
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
movies

In [ ]:
users = pd.read_csv("../data/ml-100k/u_id_mapping_demographic_.csv", sep="\t")
# users


users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
# user_features_numpy = users.to_numpy()
users

In [ ]:
dataset = rating_data
dataset

In [ ]:
ratio_split = StratifiedSplit(
    data=dataset,
    test_size=0.2,
    rating_threshold=0.0,
    seed=123,
    verbose=True,
    chrono=True,
)

hr_10 = cornac.metrics.HitRatio(k=20)
ndcg_10 = cornac.metrics.NDCG(k=20)
recall_10 = cornac.metrics.Recall(k=20)
prec_10 = cornac.metrics.Precision(k=20)
auc = cornac.metrics.AUC()
map = cornac.metrics.MAP()

epochs = [20, 40, 60, 80, 100]
models = []
# class cornac.models.vaecf.recom_vaecf.
# VAECF(name='VAECF', k=10, autoencoder_structure=[20], act_fn='tanh',
# likelihood='mult', n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, trainable=True, verbose=False, seed=None, use_gpu=False)[source]
for i in range(len(epochs)):
    models.append(
        NeuMF(
            name=f"NeuMFe={epochs[i]}",
            num_factors=8,
            layers=(64, 32, 16, 8),
            act_fn="relu",
            reg=0.0,
            num_epochs=epochs[i],
            batch_size=256,
            num_neg=4,
            lr=0.001,
            learner="adam",
            backend="tensorflow",
            early_stopping=None,
            trainable=True,
            verbose=True,
            seed=123,
        )
    )
# model=VAECF(k=10, autoencoder_structure=[20], name="vae100",act_fn="tanh", likelihood="mult", n_epochs=100, batch_size=100, learning_rate=0.001, beta=1.0, seed=123, verbose=True)

# models = [model]
cornac.Experiment(
    ratio_split, models=models, metrics=[hr_10, ndcg_10, recall_10, auc, prec_10, map]
).run()

In [ ]:
user_ids = users["userID"].to_numpy()
item_ids = movies["itemID"].to_numpy()
user_ids.__len__()

In [9]:
# get the top_k ratings for all users:
top_k = 100
reco_matrix = np.zeros((len(models), len(user_ids), top_k), dtype=int)
reco_matrix_mapped_items = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=int
)
reco_matrix_mapped_scores = np.zeros(
    (len(models), len(user_ids), len(item_ids)), dtype=float
)
reco_matrix_all = np.zeros((len(models), len(user_ids), len(item_ids)), dtype=int)


for u in user_ids:
    for i in range(len(models)):
        reco_items = models[i].recommend(u)
        items_mapped, mapped_scores = models[i].rank(
            user_idx=u, item_indices=list(item_ids)
        )
        reco_matrix_mapped_items[i][u] = items_mapped
        reco_matrix_mapped_scores[i][u] = mapped_scores
        reco_matrix_all[i][u] = reco_items
        reco_matrix[i][u] = reco_items[:top_k]

        # print(reco_matrix[0][3])

In [ ]:
hdbkhsa

In [ ]:
test_set_data = pd.DataFrame(ratio_split.test_set.uir_tuple).transpose()
test_set_data.columns = ["uid", "iid", "rating"]
test_set_data = test_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

test_set_data["uid"] = test_set_data["uid"].map(r_global_uid_map)
test_set_data["iid"] = test_set_data["iid"].map(r_global_iid_map)
test_set_data

In [13]:
train_set_data = pd.DataFrame(ratio_split.train_set.uir_tuple).transpose()
train_set_data.columns = ["uid", "iid", "rating"]
train_set_data = train_set_data.astype({"uid": "int", "iid": "int", "rating": "int"})
r_global_uid_map = {v: k for k, v in ratio_split.global_uid_map.items()}
r_global_iid_map = {v: k for k, v in ratio_split.global_iid_map.items()}

train_set_data["uid"] = train_set_data["uid"].map(r_global_uid_map)
train_set_data["iid"] = train_set_data["iid"].map(r_global_iid_map)

In [14]:
test_set_data.to_csv("testing_set_seed123_ml100k.csv", index=False, header=False)


In [15]:
train_set_data.to_csv("training_set_seed123_ml100k.csv", index=False, header=False)


In [13]:
np.save("reco_matrix_vaecf_100k_100",reco_matrix[0])


In [ ]:
reco_matrix[0].shape

In [14]:
from collections import OrderedDict
# uid_map (OrderDict, required) – The dictionary containing mapping from user original ids to mapped integer indices.

# iid_map (OrderDict, required) – The dictionary containing mapping from item original ids to mapped integer indices.
sorted_by_values = OrderedDict(sorted(models[0].iid_map.items(), key=lambda item: item[1]))
keys_sorted_by_values = list(sorted_by_values.keys())

reco_items_scores_all = [OrderedDict() for _ in range(len(user_ids))]

for u in user_ids:
    actual_index_u = u
    mapped_index_u = models[0].uid_map[actual_index_u]
    # print(f"u {u} mapped u {mapped_index_u}")
    
    # for i in item_ids:
    mapped_scores = reco_matrix_mapped_scores[0][mapped_index_u]
    ordered_dict = OrderedDict(zip(keys_sorted_by_values, mapped_scores))
    reco_items_scores_all[actual_index_u] = ordered_dict
    
    
    

In [ ]:
models[0].uid_map.items()

In [ ]:
models[0].iid_map.items()

In [ ]:
i,s=models[0].rank(98, item_ids)
print(s.__len__())
np.argsort(s)[-10:][::-1]

In [ ]:
matching_keys = [key for key, value in models[0].iid_map.items() if value == 35]
matching_keys


In [ ]:
reco_items_scores_all.__len__()

In [ ]:
import pickle
with open('score_dicts_vaecf100k.pkl', 'wb') as file:
    pickle.dump(reco_items_scores_all, file)

print("List of OrderedDicts saved to 'score_dicts.pkl'.")


In [ ]:
reco_matrix[0][0]

In [ ]:
reco_items_scores_all[98]

In [ ]:
x = OrderedDict(sorted(reco_items_scores_all[772].items(), key=lambda item: item[1], reverse=True))
x
